#Provided Data Exploration
### Adapted from https://github.com/cs109-students/michaeljohns-2015hw/blob/hw5/hw5part1.ipynb

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
## MLJ: Additional Extras
import time
import itertools

In [3]:
import os
# os.environ['PYSPARK_PYTHON'] = '/anaconda/bin/python'

In [4]:
import findspark
findspark.init()
print findspark.find()
# Depending on your setup you might have to change this line of code
#findspark makes sure I dont need the below on homebrew.
#os.environ['SPARK_HOME']="/usr/local/Cellar/apache-spark/1.5.1/libexec/"
#the below actually broke my spark, so I removed it. 
#Depending on how you started the notebook, you might need it.
# os.environ['PYSPARK_SUBMIT_ARGS']="--master local pyspark --executor-memory 4g"

/home/vagrant/spark


In [5]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local[4]')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [6]:
sc._conf.getAll()

[(u'spark.executor.memory', u'2g'),
 (u'spark.master', u'local[4]'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.memory', u'8g'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'pyspark')]

In [7]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 

In [8]:
sys.version

'2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [9]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

#Load Provided Data

In [10]:
# load the provided lyrics
lyrics_pd_df = pd.read_csv("../../data/provided/all billboard top 100 songs from 1970-2014.csv")  
# lyrics_pd_df = pd.read_csv("../../data/provided/all billboard top 100 songs from 1970-2014.csv", 
#                            names = ['<column 1>','<column 2>']) # if no header

In [11]:
lyrics_pd_df = lyrics_pd_df[['position','year','title.href','title','artist','lyrics']]

In [12]:
lyrics_pd_df.shape

(4500, 6)

In [13]:
lyrics_pd_df.head()

,position,year,title.href,title,artist,lyrics
0,1,1970,https://en.wikipedia.org/wiki/Bridge_over_Trou...,Bridge over Troubled Water,Simon and Garfunkel,When you're weary feeling small When tears are...
1,2,1970,https://en.wikipedia.org/wiki/(They_Long_to_Be...,(They Long to Be) Close to You,The Carpenters,x
2,3,1970,https://en.wikipedia.org/wiki/American_Woman_(...,American Woman,The Guess Who,"American woman, stay away from me American wom..."
3,4,1970,https://en.wikipedia.org/wiki/Raindrops_Keep_F...,Raindrops Keep Fallin' on My Head,B.J. Thomas,Raindrops keep falling on my head Just like th...
4,5,1970,https://en.wikipedia.org/wiki/War_(Edwin_Starr...,War,Edwin Starr,"War huh Yeah! Absolutely uh-huh, uh-huh huh Ye..."


In [14]:
# convert from pandas to spark dataframe
lyricsdf = sqlsc.createDataFrame(lyrics_pd_df)

In [15]:
# view output
lyricsdf.show(3)

+--------+----+--------------------+--------------------+-------------------+--------------------+
|position|year|          title.href|               title|             artist|              lyrics|
+--------+----+--------------------+--------------------+-------------------+--------------------+
|       1|1970|https://en.wikipe...|Bridge over Troub...|Simon and Garfunkel|When you're weary...|
|       2|1970|https://en.wikipe...|(They Long to Be)...|     The Carpenters|                   x|
|       3|1970|https://en.wikipe...|      American Woman|      The Guess Who|American woman, s...|
+--------+----+--------------------+--------------------+-------------------+--------------------+
only showing top 3 rows



In [16]:
# no longer need the pandas version, clear it out
del lyrics_pd_df

In [17]:
from pyspark.sql import functions as F
"""
add a `song_key` column by joining `year` and `position` for better identity 
"""
##################################################################################################
# CITATION - Concat usage within Spark Dataframe adapted from:
# http://stackoverflow.com/questions/31450846/concatenate-columns-in-apache-spark-dataframe
##################################################################################################
lyricsdf = lyricsdf.withColumn('song_key',F.concat(F.col("year"), F.lit("-"), F.col("position")))

In [18]:
#view output
lyricsdf.show(3)

+--------+----+--------------------+--------------------+-------------------+--------------------+--------+
|position|year|          title.href|               title|             artist|              lyrics|song_key|
+--------+----+--------------------+--------------------+-------------------+--------------------+--------+
|       1|1970|https://en.wikipe...|Bridge over Troub...|Simon and Garfunkel|When you're weary...|  1970-1|
|       2|1970|https://en.wikipe...|(They Long to Be)...|     The Carpenters|                   x|  1970-2|
|       3|1970|https://en.wikipe...|      American Woman|      The Guess Who|American woman, s...|  1970-3|
+--------+----+--------------------+--------------------+-------------------+--------------------+--------+
only showing top 3 rows



In [19]:
#We cache the data to make sure it is only read once from disk
lyricsdf.cache()
print "How many songs do we have?", lyricsdf.count()

How many songs do we have? 4500


In [20]:
print "What is the schema?", lyricsdf.printSchema()

What is the schema? root
 |-- position: long (nullable = true)
 |-- year: long (nullable = true)
 |-- title.href: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- lyrics: string (nullable = true)
 |-- song_key: string (nullable = true)

None


#Sample

Some initial sampling to take from each year.

In [21]:
# whether or not to sample lyrics, and how many to sample per year
sample_lyrics = True
PER_YEAR_SAMPLES=10

In [22]:
#(your code here)
def randomSubSampleLyrics(sparkdf,take=PER_YEAR_SAMPLES):    
    # generate spark pairs as a tuple
    br_pairs = sparkdf.map(lambda r: (r.year, r.song_key))
    
    # group by key for a list of reviews per business and collect
    br_grouped = br_pairs.groupByKey().mapValues(lambda x: list(x)).collect()
        
    #sample after collect
    br_sample = [np.random.choice(v, size=take, replace=False) for k,v in br_grouped]    
    
    #flatten into a list
    return list(itertools.chain.from_iterable(br_sample))
    
small_song_keys = randomSubSampleLyrics(lyricsdf)

In [23]:
if sample_lyrics:
    print "How many small_song_keys? ", len(small_song_keys)
    small_song_keys[:5]
else:
    print "No lyric sampling, full processing (change `sample_lyrics` value to `True` to sample)"

How many small_song_keys?  450


#Sample Lyrics (or Not)

In [24]:
print "execution start --> {}".format(time.strftime('%a, %d %b %Y %H:%M:%S', time.localtime()))

execution start --> Wed, 18 Nov 2015 13:44:33


In [25]:
%%time
#(your code here)
if sample_lyrics:
    ldf=lyricsdf[lyricsdf.song_key.isin(small_song_keys)]#creates new dataframe
else:
    ldf=lyricsdf

CPU times: user 0 ns, sys: 138 ms, total: 138 ms
Wall time: 486 ms


In [26]:
# cache results
ldf.cache()

DataFrame[position: bigint, year: bigint, title.href: string, title: string, artist: string, lyrics: string, song_key: string]

In [27]:
print "How many lyrics are in ldf? ", ldf.count()

How many lyrics are in ldf?  450


#NLP

In [28]:
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')

In [29]:
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

In [30]:
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

In [31]:
print "Quick Test of parse..."
parse("The world is the craziest place. I am working hard.", tokenize=True, lemmata=True)

Quick Test of parse...


u'The/DT/B-NP/O/the world/NN/I-NP/O/world is/VBZ/B-VP/O/be the/DT/B-NP/O/the craziest/JJ/I-NP/O/craziest place/NN/I-NP/O/place ././O/O/.\nI/PRP/B-NP/O/i am/VBP/B-VP/O/be working/VBG/I-VP/O/work hard/RB/B-ADVP/O/hard ././O/O/.'

In [32]:
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    return nouns2, descriptives2

In [33]:
print "Quick check of get_parts ..."
get_parts("Have had many other items and just love the food. The patio...job was and...perfect. Lunch is good, and the only egg is great")

Quick check of get_parts ...


([[u'patio', u'job'], [u'lunch', u'egg']], [[u'perfect'], [u'good', u'great']])

###Run Get Parts on Provided Data

In [34]:
#(your code here)
lyric_parts = ldf.map(lambda r : get_parts(r.lyrics))

In [35]:
# view output
lyric_parts.take(2)

[([[u'question'],
   [u'party',
    u'light',
    u'way',
    u'fun',
    u'window',
    u'air',
    u'room',
    u'smell',
    u'perfume',
    u'cigarette',
    u'smoking',
    u'half',
    u'death',
    u'window',
    u'sucker',
    u'breath',
    u'way',
    u'fun',
    u'son',
    u'way',
    u'fun',
    u'son',
    u'radio',
    u'door',
    u'girlfriend',
    u'floor',
    u'thing',
    u'way',
    u'fun',
    u'son',
    u'way',
    u'fun',
    u'way',
    u'fun',
    u'way',
    u'fun',
    u'son',
    u'way',
    u'fun',
    u'way',
    u'fun',
    u'son',
    u'way',
    u'fun',
    u'way',
    u'fun',
    u'son',
    u'way',
    u'fun',
    u'way',
    u'fun',
    u'son']],
  [[u'crazy'], [u'craziest', u'stale']]),
 ([[u'thing', u'time', u'thing'], [u'love', u'girl']],
  [[u'little'], [u'short', u'right', u'wrong']])]

In [36]:
print "execution start --> {}".format(time.strftime('%a, %d %b %Y %H:%M:%S', time.localtime()))

execution start --> Wed, 18 Nov 2015 13:44:35


In [37]:
%%time
parseout=lyric_parts.collect()

CPU times: user 10.7 ms, sys: 2.77 ms, total: 13.5 ms
Wall time: 5.9 s


#Vocab
##Nouns

In [38]:
print "How many parseout entries? ", len(parseout)

How many parseout entries?  450


In [39]:
# flatten parseout to create initial noun rdd
nounrdd=sc.parallelize([ele[0] for ele in parseout]).flatMap(lambda l: l)

In [58]:
# view output
nounrdd.take(1)

[[u'question']]

In [41]:
# cache results
nounrdd.cache()

PythonRDD[43] at RDD at PythonRDD.scala:43

In [42]:
# collect all the words and cache
nounvocabtups = (nounrdd.flatMap(lambda word: word)
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
             .map(lambda (x,y): x)
             .zipWithIndex()
)

In [43]:
# view output
nounvocabtups.take(3)

[(u'shop', 0), (u'liar', 1), (u'dance', 2)]

In [44]:
# cache results
nounvocabtups.cache()

PythonRDD[50] at RDD at PythonRDD.scala:43

In [45]:
# collect results
nounvocab=nounvocabtups.collectAsMap()
nounid2word=nounvocabtups.map(lambda (x,y): (y,x)).collectAsMap()

In [46]:
# since sampling may be used, avoiding more common usage, e.g. `nounvocab['dance']`
nounid2word[0], nounvocab.keys()[5], nounvocab[nounvocab.keys()[5]]

(u'shop', u'hate', 182)

In [47]:
print "How big is the noun vocabulary? ", len(nounvocab.keys())

How big is the noun vocabulary?  2361


##Adjectives

In [48]:
# create initial adj rdd from parseout
adjrdd=sc.parallelize([ele[1] for ele in parseout])

In [49]:
# view output
adjrdd.take(3)

[[[u'crazy'], [u'craziest', u'stale']],
 [[u'little'], [u'short', u'right', u'wrong']],
 [[u'warm'],
  [u'rainy',
   u'rainy',
   u'distant',
   u'sad',
   u'rainy',
   u'rainy',
   u'hard',
   u'rainy',
   u'rainy',
   u'lonely',
   u'lonely'],
  [u'rainin']]]

In [50]:
# cache results
adjrdd.cache()

ParallelCollectionRDD[52] at parallelize at PythonRDD.scala:423

In [51]:
#(your code here)
adjvocabtups = (adjrdd.flatMap(lambda l: l)
              .flatMap(lambda word: word)
              .map(lambda word: (word, 1))
              .reduceByKey(lambda a, b: a + b)
              .map(lambda (x,y): x)
              .zipWithIndex()
)

In [52]:
# view output
adjvocabtups.take(3)

[(u'exotic', 0), (u'shot', 1), (u'lean', 2)]

In [53]:
# cache results
adjvocabtups.cache()

PythonRDD[60] at RDD at PythonRDD.scala:43

In [54]:
# collect results
adjvocab=adjvocabtups.collectAsMap()
adjid2word=adjvocabtups.map(lambda (x,y): (y,x)).collectAsMap()

In [55]:
# since sampling may be used, avoiding more common usage, e.g. `adjvocab['exotic']`
adjid2word[0], adjvocab.keys()[5], adjvocab[adjvocab.keys()[5]]

(u'exotic', u'two-three', 6)

In [56]:
print "How big is the adjective vocabulary? ", len(adjvocab)

How big is the adjective vocabulary?  763


# BOW Corpus

In [59]:
##################################################################################################
# CITATION - Use of counter for reduce within each word list from:
# http://stackoverflow.com/questions/2600191/how-can-i-count-the-occurrences-of-a-list-item-in-python
##################################################################################################
from collections import Counter

# for each sentence, reduct into a list of tuple k,v where k=vocab index and v=count, 
# each word list is sorted by occurence
documents = nounrdd.map(lambda words: Counter([nounvocab[word] for word in words]).most_common())

In [61]:
# verify output
documents.take(3)

[[(539, 1)],
 [(1284, 13),
  (407, 13),
  (2281, 7),
  (1559, 2),
  (769, 1),
  (386, 1),
  (870, 1),
  (742, 1),
  (583, 1),
  (1000, 1),
  (697, 1),
  (97, 1),
  (974, 1),
  (2097, 1),
  (2222, 1),
  (2169, 1),
  (721, 1),
  (572, 1),
  (2215, 1),
  (2334, 1),
  (309, 1)],
 [(721, 2), (2359, 1)]]

In [62]:
# gather spark results
corpus=documents.collect()